In [1]:
import numpy as np
from PIL import Image
import os
import keras
import random

Using TensorFlow backend.


In [17]:
path = 'Trainset/Trainset'
f = os.listdir(path)
random.shuffle(f)


def generate_arrays_from_file(f,batch_size):
    while 1:
        cnt = 0
        X =[]
        Y =[]
        for line in f:
            if line[:2] == '00':
                img = Image.open(path+'/'+line)
                img = img.convert('L')
                img = img.resize((400,400),Image.ANTIALIAS)
                Y.append(0)
                X.append(np.array(img)/255)
            elif line[:2]=='10':
                img = Image.open(path+'/'+line)
                img = img.convert('L')
                img = img.resize((400,400),Image.ANTIALIAS)
                X.append(np.array(img)/255)
                Y.append(1)
            cnt += 1
            if cnt==batch_size:
                cnt = 0
                yield (np.array(X).reshape(batch_size,160000), np.array(Y).reshape(-1,1))    
                X = []
                Y = []

            
            
# generate_arrays_from_file('Trainset/Trainset',64)

In [8]:
from keras.layers import Input, Dense,Conv1D,Activation,MaxPooling1D,Flatten,Dropout
from keras.models import Model
from keras import optimizers
# shape = (400,400,3)

model = keras.Sequential()
# # # a layer instance is callable on a tensor, and returns a tensor
# model.add(Dense(1024, activation='sigmoid',input_dim=160000))
# model.add(Dense(512,activation='sigmoid'))
# model.add(Dense(256,activation='sigmoid'))
# model.add(Dense(64, activation='sigmoid'))
# model.add(Dropout(0.5))
# this creates a model that include\s
# # the Input layer and three Dense layers
model.add(Conv1D(32, 3,padding="same",input_shape=(400,400)))
model.add(Activation("relu"))
model.add(MaxPooling1D(pool_size=2, strides=2))
#second set of CONV => RELU => POOL layers
model.add(Conv1D(32, 3, padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling1D(pool_size=2, strides=2))
# first (and only) set of FC => RELU layers

model.add(Conv1D(64, 3, padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling1D(pool_size=2, strides= 2))

model.add(Dropout(0.25))
# model.add(Conv1D(50, (5, 5), padding="same"))
# model.add(Activation("sigmoid"))
# model.add(MaxPooling1D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Activation("relu"))
model.add(Dense(64))
model.add(Activation("relu"))

        # softmax classifier
model.add(Dense(8,activation='relu'))
model.add(Dense(1,name='this'))
model.add(Activation("sigmoid"))


# sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit_generator(generate_arrays_from_file(f[:10000],16),samples_per_epoch=10000,epochs=10,
                   validation_data=generate_arrays_from_file(f[10000:],16),validation_steps=16)



/home/xms/.virtualenvs/dl/lib/python3.5/site-packages/ipykernel_launcher.py:50: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=10000, validation_steps=16, validation_data=<generator..., epochs=10)`


Epoch 1/10
   25/10000 [..............................] - ETA: 1:22:13 - loss: 0.6950 - acc: 0.5200

/home/xms/.virtualenvs/dl/lib/python3.5/site-packages/PIL/Image.py:966: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


  147/10000 [..............................] - ETA: 1:09:46 - loss: 0.6938 - acc: 0.5272

KeyboardInterrupt: 

In [45]:

layer_model = Model(inputs=model.input,outputs=model.get_layer('this').output)

def prediction_cla(file):
    data_list = os.listdir(file)
    for i in data_list:
        if i[:2] == '20':
            img = Image.open(file+'/'+i)
            img = img.convert('L')
            img = img.resize((200,200),Image.ANTIALIAS)
            img = np.array([np.array(img)])
            result = model.predict_proba(img)
#             print(result)
#             print(model.predict_proba(img))
            with open('reuslt.csv','a+') as f:
                if data_list.index(i) == 0:
                    f.write('pic_id,pred'+'\n')
                    f.write('pic_'+i.replace('.jpg','')+','+str(result[0][0])+'\n')
                else:
                    f.write('pic_'+i.replace('.jpg','')+','+str(result[0][0])+'\n')

    
prediction_cla('智慧环保初赛测试集及提交样例/智慧环保初赛测试集及提交样例/Test_A')

In [8]:
def generate_arrays_from_fil(f,batch_size):
    while 1:
        cnt = 0
        X =[]
        Y =[]
        for line in f:
            if line[:2] == '00':
                img = Image.open(path+'/'+line)
                img = img.convert('RGB')
                img = img.resize((200,200),Image.ANTIALIAS)
#                 print(np.array(img).shape)
                Y.append(0)
                X.append(np.array(img)/255)
            elif line[:2]=='10':
                img = Image.open(path+'/'+line)
                img = img.convert('RGB')
                img = img.resize((200,200),Image.ANTIALIAS)
                X.append(np.array(img)/255)
                Y.append(1)
            cnt += 1
            if cnt==batch_size:
                cnt = 0
                yield (np.array(X), np.array(Y).reshape(-1,1))    
                X = []
                Y = []

In [21]:
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Input, Dense,Conv1D,Activation,MaxPooling1D,Flatten,Dropout
from keras.models import Model
model = keras.Sequential()
# # # a layer instance is callable on a tensor, and returns a tensor
# model.add(Dense(1024, activation='sigmoid',input_dim=160000))
# model.add(Dense(512,activation='sigmoid'))
# model.add(Dense(256,activation='sigmoid'))
# model.add(Dense(64, activation='sigmoid'))
# model.add(Dropout(0.5))
# this creates a model that include\s
# # the Input layer and three Dense layers
model.add(Conv2D(128, (5,5),input_shape=(200,200,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

#second set of CONV => RELU => POOL layers
model.add(Conv2D(128, (5,5)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
# first (and only) set of FC => RELU layers

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2), strides= (2,2)))

model.add(Conv2D(32, (3,3)))
model.add(Activation("relu"))
model.add(Conv2D(32, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2), strides= (2,2)))


# model.add(Conv1D(50, (5, 5), padding="same"))
# model.add(Activation("sigmoid"))
# model.add(MaxPooling1D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.8))
model.add(Dense(128))
model.add(Activation("relu"))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(4,activation='relu'))
model.add(Dense(1,name='this'))
model.add(Activation("sigmoid"))


# sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
print(model.summary())
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit_generator(generate_arrays_from_fil(f[:100],1),verbose=1,samples_per_epoch=100,epochs=10,
                   validation_data=generate_arrays_from_fil(f[100:120],1),validation_steps=1)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_52 (Conv2D)           (None, 196, 196, 128)     9728      
_________________________________________________________________
activation_77 (Activation)   (None, 196, 196, 128)     0         
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 97, 97, 128)       0         
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 93, 93, 128)       409728    
_________________________________________________________________
activation_78 (Activation)   (None, 93, 93, 128)       0         
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 46, 46, 128)       0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 44, 44, 64)        73792     
__________

/home/xms/.virtualenvs/dl/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=1, verbose=1, validation_data=<generator..., steps_per_epoch=100, epochs=10)`


Epoch 1/10
100/100 [==============================] - 71s 711ms/step - loss: 0.6956 - acc: 0.4500 - val_loss: 0.7051 - val_acc: 0.0000e+00
Epoch 2/10
100/100 [==============================] - 74s 737ms/step - loss: 0.6945 - acc: 0.5100 - val_loss: 0.6808 - val_acc: 1.0000
Epoch 3/10
 80/100 [=======================>......] - ETA: 14s - loss: 0.6931 - acc: 0.5250

KeyboardInterrupt: 

In [25]:
minibatch_test_iterators = generate_arrays_from_file(f[10000:],128)
X_test, y_test = minibatch_test_iterators.__next__()  # 得到一份测试文件
from sklearn.linear_model import SGDClassifier,LinearRegression
sgd_clf = SGDClassifier()  

minibatch_train_iterators = generate_arrays_from_file(f[:10000], 128)

for i, (X_train, y_train) in enumerate(minibatch_train_iterators):
    # 使用 partial_fit ，并在第一次调用 partial_fit 的时候指定 classes
    sgd_clf.partial_fit(X_train, y_train, classes=np.array([0, 1]))
    print("{} time".format(i))  # 当前次数
    print("{} train_score".format(sgd_clf.score(X_train,y_train)))
    print("{} score".format(sgd_clf.score(X_test, y_test)))  # 在测试集上看效果


Exception ignored in: <generator object generate_arrays_from_file at 0x7febc5159780>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object generate_arrays_from_file at 0x7febc4e438e0>
RuntimeError: generator ignored GeneratorExit
/home/xms/.virtualenvs/dl/lib/python3.5/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0 time
0.6015625 train_score
0.5 score


/home/xms/.virtualenvs/dl/lib/python3.5/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1 time
0.4453125 train_score
0.5078125 score


/home/xms/.virtualenvs/dl/lib/python3.5/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


2 time
0.5859375 train_score
0.5 score


/home/xms/.virtualenvs/dl/lib/python3.5/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


3 time
0.6640625 train_score
0.4609375 score


KeyboardInterrupt: 